In [1]:
import numpy as np
import pandas as pd

### Get Iris Dataset online

In [2]:
uri = "https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv"
columns = ["sepal_length","sepal_width","petal_length","petal_width","type"]
dataset = pd.read_csv(uri, skiprows=1, names=columns)

In [3]:
dataset

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [4]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None,
                 info_gain=None, value=None):
        
        # attr of Decision nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # attr of leaf nodes
        self.value = value

In [14]:
class DecisionTreeClassifier:
    def __init__(self, split_min_samples=2, max_depth=2):
        
        self.root = None
        
        # stopping hyper-parameters
        self.split_min_samples = split_min_samples
        self.max_depth = max_depth
        
        
    def buildTree(self, dataset, current_depth=0):
        
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = X.shape
        
        if num_samples >= self.split_min_samples and current_depth <= self.max_depth:
            
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            if best_split["info_gain"] > 0:
                left_subtree = self.buildTree(best_split['dataset_left'], current_depth+1)
                right_subtree = self.buildTree(best_split['dataset_right'], current_depth+1)
                return Node(best_split['feature_index'], best_split['threshold'],
                           left_subtree, right_subtree, best_split['info_gain'])
            
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
                
        
    def get_best_split(self, dataset, num_samples, num_features):
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            
            feature_col = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_col)
            
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, y_left, y_right = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    current_info_gain = self.information_gain(y, y_left, y_right, "gini")
                    if current_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = current_info_gain
                        max_info_gain = current_info_gain
                        
        return best_split
                        
    def split(self, dataset, feature_index, threshold):
        feature_col = dataset[:,feature_index]
        dataset_left = np.array([ row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right
    
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child)/len(parent)
        weight_r = len(r_child)/len(parent)
        
        if mode == "gini":
            gain = self.gini(parent) - ( weight_l*self.gini(l_child) + weight_r*self.gini(r_child) )
        else:
            gain = self.entropy(parent) - ( weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child) )
            
        return gain
    
    
    def entropy(self, y):
        feature_classes = np.unique(y)
        entropy = 0
        
        for _class in feature_classes:
            class_probability = len(y[y==_class])/len(y) 
            entropy += -class_probability*np.log2(class_probability)
            
        return entropy
    
    
    def gini(self, y):
        feature_classes = np.unique(y)
        gini = 0
        
        for _class in feature_classes:
            class_probability = len(y[y==_class])/len(y)
            gini += class_probability**2
            
        return 1 - gini
    
    
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    
    def print_tree(self, tree=None, indent="\t"):
        
        if not tree:
            tree = self.root
        if tree.value:
            print(tree.value)
        else:
            print(f'x_{tree.feature_index} <= {tree.threshold}? information_gain = {tree.info_gain:.4f}')
            print(f'{indent}left: ', end="")
            self.print_tree(tree.left, indent+"  ")
            print(f'{indent}right: ', end="")
            self.print_tree(tree.right, indent+"  ")
            
            
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.buildTree(dataset)
        
        
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    
    def make_prediction(self, x, tree):
        if tree.value: 
            return tree.value
        feature_value = x[tree.feature_index]
        if feature_value <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [15]:
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values.reshape(-1, 1)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=54)

In [17]:
d_tree_classifier = DecisionTreeClassifier(split_min_samples=3, max_depth=3)
d_tree_classifier.fit(X_train, Y_train)
d_tree_classifier.print_tree()

x_2 <= 1.9? information_gain = 0.3412
	left: Setosa
	right: x_3 <= 1.7? information_gain = 0.4038
	  left: x_2 <= 5.1? information_gain = 0.0881
	    left: x_0 <= 4.9? information_gain = 0.0243
	      left: Virginica
	      right: Versicolor
	    right: Virginica
	  right: x_2 <= 4.8? information_gain = 0.0166
	    left: x_0 <= 5.9? information_gain = 0.4444
	      left: Versicolor
	      right: Virginica
	    right: Virginica


In [28]:
y_predicted = np.array(d_tree_classifier.predict(X_test)).reshape(-1,1)

In [34]:
accuracy = len(Y_test[Y_test == y_predicted])/ len(Y_test)
accuracy

0.9333333333333333